# Polymorphic Berman Quartz

A notebook to generate spud files for a polymorphic Endmember for B-quartz and B-cristobalite
that returns 

$$
\min(G_{qz},G_{cr})
$$

where 
* $G_{bQz}$ is the berman model for the Standard state Gibbs free energy of B-quartz
* $G_{bCr}$ is the berman model for the Standard state Gibbs free energy of beta-cristobalite

Required system packages and initialization

In [ ]:
import os,sys
import pandas as pd
import numpy as np
import sympy as sym
import time
sym.init_printing()

Required ENKI packages

In [ ]:
from thermocodegen.coder import coder

### let's set up some directory names for clarity

In [ ]:
HOME_DIR = os.path.abspath('../../')
SPUD_DIR = HOME_DIR+'/endmembers'
try:
    os.makedirs(SPUD_DIR)
except:
    pass

Set a reference string for this Notebook

In [ ]:
reference = 'fo_sio2/notebooks/Generate_Polymorphic-Quartz.ipynb'

## Clean the Berman Data base for just the SiO2 polymorphs

* Here we'll just use  standard state models without lambda transitions or order-disorder terms


In [ ]:
df = pd.read_json('data/berman_1988.json')
df.columns

In [ ]:
# pull out column names required for just standard state model
cols_ss = ['Phase', 'Formula', 'sAbbrev', 'H_TrPr', 'S_TrPr', 'V_TrPr',
           'k0', 'k1', 'k2', 'k3', 'v1', 'v2', 'v3', 'v4']

Just pull out all SiO2 polymorphs and clean up columns


In [ ]:
df_si = df[df['Formula']=='SI(1)O(2)']
df_si = df_si[cols_ss]
df_si = df_si.rename(index=str,columns={'Phase':'name', 'Formula':'formula'})
df_si.set_index('name')

Choose Polymorphs of interest

In [ ]:
polymorphs = [ 'BETA_CRISTOBALITE', 'B-QUARTZ']
df_poly = df_si.set_index('name').filter(items=polymorphs,axis=0)
df_poly

## Generate Sympy symbols and Coder parameters from the column headers

To construct our Gibbs free energy we need to create Sympy symbols and units for the parameters
* H_TrPr: Enthalpy at reference $T$ and $P$ in $J$
* S_TrPr: Entropy at reference T and P in $J/K$
* V_TrPr: Volume at reference T and P in $J/bar$
* k0-k3:  Coefficients in Berman heat capacity model with units [ 'J/K-m', 'J-K^(1/2)-m', 'J-K/m', 'J-K^2' ]
* v1-v4:  Coefficients in Berman Equation of state (V) with units [ '1/bar', '1/bar^2' , '1/K', '1/K^2']



We start with the root names/symbols for parameters

In [ ]:
param_roots = df_poly.columns[2:]
base_units = [ 'J', 'J/K', 'J/bar-m',
         'J/K-m','J-K^(1/2)-m','J-K/m' ,'J-K^2',
        '1/bar', '1/bar^2', '1/K',  '1/K^2']


Add root symbols to local data base

In [ ]:
params_root = coder.set_coder_params(param_roots, base_units)
symbol_dict_root = coder.get_symbol_dict_from_params(params_root)

# load local dictionary
locals().update(symbol_dict_root)
symbol_dict_root

But because we are combining two different endmembers into a single pseudo-phase we will need to append our parameter names with the subspecies abbreviation

In [ ]:
param_strings = []
for index, row in df_poly.iterrows():
    abbrev = row['sAbbrev']
    param_strings += [ '{}_{}'.format(p,abbrev) for p in param_roots]
param_strings 

#### Add units and generate parameter tuples needed for coder model

In [ ]:
units = base_units + base_units

Inspect parameters and  units

In [ ]:
print('parameter\tunit')
for t in list(zip(param_strings, units)):
    print('{}\t\t {}'.format(t[0],t[1]))

We now construct coder parameters and sympy symbols from the parameter names and pass them to the local dictionary for use in other sympy expressions

In [ ]:
params = coder.set_coder_params(param_strings, units)
symbol_dict = coder.get_symbol_dict_from_params(params)

# load local dictionary
locals().update(symbol_dict)

## Create  model class

In [ ]:
model_ss = coder.StdStateModel.from_type()

Retrieve sympy symbols for model variables and reference conditions

In [ ]:
T = model_ss.get_symbol_for_t()
P = model_ss.get_symbol_for_p()
Tr = model_ss.get_symbol_for_tr()
Pr = model_ss.get_symbol_for_pr()

## Standard State Potentials

### Define model Potentials for the Standard State Potentials
An expression for the Gibbs free energy, $G(T,P)$ or the Helmholtz energy $A(T,V)$ is constructed.  The expression may have multiple parts.  Often the heat capacity function is postulated, then integrated to yield expressions for the entahlpy, entropy, and in combination the energy potential. Then, an equation of state (EOS) is adopted and that term is integrated in pressure or volume and added to the heat capacity integrals. This proceedure is follwed here.
#### (1) $C_P$ integrals
The isobaric heat capacity terms parameterized as: $C_P = k_0 + k_1 / T^{1/2} + k_2 / T^2 + k_3 / T^3 $,
and in addition the reference condition third law entropy, $ S_{Tr,Pr} $, and enthalpy of formation from the
elements, $ \Delta H_{Tr,Pr} $, constitute additional parameters:

In [ ]:
# Heat Capacity
CpPr = k0+k1/sym.sqrt(T)+k2/T**2+k3/T**3

Define the heat capacity contribution to the Gibbs free energy ...

In [ ]:
GPr = H_TrPr + sym.integrate(CpPr,(T,Tr,T)) - T*(S_TrPr + sym.integrate(CpPr/T,(T,Tr,T)))

#### (2) $V$ (EOS) integrals
Next, define a volume-explicit equation of state applicable over the whole of temperature and pressure space

In [ ]:
V = V_TrPr*(1+v1*(P-Pr)+v2*(P-Pr)**2+v3*(T-Tr)+v4*(T-Tr)**2)

EOS contributions to the gibbs free energh

In [ ]:
GPrToP = sym.integrate(V,(P,Pr,P))

#### Define generic standard state Gibbs model

In [ ]:
G_ss = GPr + GPrToP
G_ss

####  generate specific G for each end_member


In [ ]:
abbrevs=df_poly['sAbbrev'].tolist()
N = len(param_roots)
G_p = [ sym.Symbol('G_{}'.format(a)) for a in abbrevs]

In [ ]:
symbol_list = list(symbol_dict.values())
for i,p in enumerate(polymorphs):
    subs_dict = dict(zip(symbol_dict_root.values(),symbol_list[i*N:(i+1)*N]))
    G_p[i]=G_ss.subs(subs_dict)

In [ ]:
G = sym.Min(G_p[0],G_p[1])
G

### Generate a coder model and write to spud files

In [ ]:
model_ss.add_potential_to_model('G',G, params)

Exam settable values for the model dictionary

In [ ]:
values_dict = model_ss.get_values()
values_dict

populate the values dictionary

In [ ]:
values_dict['name'] = 'Silica_polymorph_berman'
values_dict['reference'] = reference
for ind, row in df_poly.iterrows():
    abbrev = row.pop('sAbbrev')
    new_columns = [ '{}_{}'.format(p,abbrev) for p in param_roots ]
    row.rename(index=dict(zip(param_roots,new_columns)),inplace=True)
    values_dict.update(row.to_dict())
values_dict 

### Write out spud files for all Berman standard state endmembers

In [ ]:
os.chdir(SPUD_DIR)
print(values_dict['name'])
model_ss.set_values(values_dict)
model_ss.to_xml(path=SPUD_DIR)